# VER GPU

In [ ]:
!nvidia-smi
import torch

if torch.cuda.is_available():
    print("GPU disponible")
else:
    print("GPU no disponible")

# IMPORTAR JSON

In [1]:
from custom_transforms import *
from dataset import *
from generarModelo import *


izquierda_dir = r"C:/Users/Alejandro/TFG_ENTRENAMIENTO/JSON_FORMADOS/IZQUIERDA_NORMALIZADO"
derecha_dir = r"C:/Users/Alejandro/TFG_ENTRENAMIENTO/JSON_FORMADOS/DERECHA_NORMALIZADO"

porcentaje_entrenamiento = 0.70
porcentaje_validacion = 0.15
porcentaje_prueba = 0.15

datos_entrenamiento, datos_validacion, datos_prueba, etiquetas_entrenamiento, etiquetas_validacion, etiquetas_prueba = cargar_y_preparar_datos(izquierda_dir, derecha_dir, porcentaje_entrenamiento, porcentaje_validacion, porcentaje_prueba)

Loading data: 100%|██████████| 7694/7694 [00:20<00:00, 369.45it/s]


# TRANSFORMAR

In [2]:
#Generar transformacion TAMAÑO
datos_entrenamiento_transformados_zoom = transformacion_zoom(datos_entrenamiento)
etiquetas_entrenamiento_transformados_zoom = etiquetas_entrenamiento

#Generar transformacion TRANSLACION EJE X
datos_entrenamiento_transformados_translacionX = desplazar_posesX(datos_entrenamiento)
etiquetas_entrenamiento_transformados_translacionX = etiquetas_entrenamiento

#Generar transformacion TRANSLACION EJE Y
datos_entrenamiento_transformados_translacionY = desplazar_posesY(datos_entrenamiento)
etiquetas_entrenamiento_transformados_translacionY = etiquetas_entrenamiento

#Generar transformacion FLIP
datos_entrenamiento_transformados_flip = flip_poses(datos_entrenamiento)
etiquetas_entrenamiento_transformados_flip = invertir_etiquetas(etiquetas_entrenamiento)

In [3]:
#Generar transformacion TAMAÑO
datos_validacion_transformados_zoom = transformacion_zoom(datos_validacion)
etiquetas_validacion_transformados_zoom = etiquetas_validacion

#Generar transformacion TRANSLACION EJE X
datos_validacion_transformados_translacionX = desplazar_posesX(datos_validacion)
etiquetas_validacion_transformados_translacionX = etiquetas_validacion

#Generar transformacion TRANSLACION EJE Y
datos_validacion_transformados_translacionY = desplazar_posesY(datos_validacion)
etiquetas_validacion_transformados_translacionY = etiquetas_validacion

#Generar transformacion FLIP
datos_validacion_transformados_flip = flip_poses(datos_validacion)
etiquetas_validacion_transformados_flip = invertir_etiquetas(etiquetas_validacion)

In [4]:
#Generar transformacion TAMAÑO
datos_prueba_transformados_zoom = transformacion_zoom(datos_prueba)
etiquetas_prueba_transformados_zoom = etiquetas_prueba

#Generar transformacion TRANSLACION EJE X
datos_prueba_transformados_translacionX = desplazar_posesX(datos_prueba)
etiquetas_prueba_transformados_translacionX = etiquetas_prueba

#Generar transformacion TRANSLACION EJE Y
datos_prueba_transformados_translacionY = desplazar_posesY(datos_prueba)
etiquetas_prueba_transformados_translacionY = etiquetas_prueba

#Generar transformacion FLIP
datos_prueba_transformados_flip = flip_poses(datos_prueba)
etiquetas_prueba_transformados_flip = invertir_etiquetas(etiquetas_prueba)

# COMPARAR ANTES Y DESPUÉS TRANSFORMACION

In [ ]:
# ZOOM
transformacion_dato(datos_entrenamiento, etiquetas_entrenamiento, datos_entrenamiento_transformados_zoom, etiquetas_entrenamiento_transformados_zoom)

In [ ]:
# TRANSLACION X
transformacion_dato(datos_entrenamiento, etiquetas_entrenamiento, datos_entrenamiento_transformados_translacionX, etiquetas_entrenamiento_transformados_translacionX)

In [ ]:
# TRANSLACION Y
transformacion_dato(datos_entrenamiento, etiquetas_entrenamiento, datos_entrenamiento_transformados_translacionY, etiquetas_entrenamiento_transformados_translacionY)

In [ ]:
# FLIP
transformacion_dato(datos_entrenamiento, etiquetas_entrenamiento, datos_entrenamiento_transformados_flip, etiquetas_entrenamiento_transformados_flip)

# VER TENSORES TRANSFORMADOS

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[1]), gif(datos_entrenamiento_transformados_zoom[1]), "resultado.gif")

Image("resultado.gif")


In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[2]), gif(datos_entrenamiento_transformados_translacionX[2]), "resultado.gif")

Image("resultado.gif")


In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[2]), gif(datos_entrenamiento_transformados_translacionY[2]), "resultado.gif")

Image("resultado.gif")


In [ ]:
from gifComparaTransformado import *
from generar_gif import *

from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[2]), gif(datos_entrenamiento_transformados_flip[2]), "resultado.gif")

Image("resultado.gif")

In [ ]:
from generar_gif import gif
from IPython.display import Image

Image(gif(datos_entrenamiento_transformados_flip[1]))

In [ ]:
print(datos_entrenamiento.shape)

# VER LOGS

In [5]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs/fino --port 64569

# ENTRENAR OPTUNA

In [ ]:
train_loader, val_loader = preparar_entrenamiento(datos_entrenamiento, etiquetas_entrenamiento, datos_validacion, etiquetas_validacion, 64)
# Entrenamiento con Optuna y guardado de logs
study = entrenar_con_optuna(train_loader, val_loader, n_trials=5, num_epochs=50, patience=5)
log_dir = "./logs/logsOptuna"
guardar_logs(study, log_dir)

# ENTRENAR FINAL

In [ ]:
print(study.best_trial.params)

In [ ]:
# Concatenar los datos de entrenamiento y validación
datos_entrenamiento_final = torch.cat((datos_entrenamiento, datos_validacion), dim=0)
etiquetas_entrenamiento_final = torch.cat((etiquetas_entrenamiento, etiquetas_validacion), dim=0)

# Preparar los dataloaders con los datos combinados
train_loader, val_loader = preparar_entrenamiento(datos_entrenamiento_final, etiquetas_entrenamiento_final, datos_prueba, etiquetas_prueba, 64)

# Entrenamiento del modelo con los mejores parámetros
best_params = study.best_trial.params
best_params_local = {'learning_rate': 8.492389446863165e-06, 'lstm_units': 1226, 'dense_units': 1094, 'dropout_rate': 0.2860955821910263}
modelo_final = entrenar_modelo(train_loader, val_loader, best_params_local, num_epochs=200)
log_dir = "./logs/logsFino"
guardar_logs(study, log_dir)


In [ ]:
# GENERAR DATASET CON TRANSFORMACIONES

In [7]:
print('ENTRENAMIENTO INICIAL',datos_entrenamiento.shape[0])
print('VALIDACION INICIAL',datos_validacion.shape[0])
print('PRUEBA INICIAL',datos_prueba.shape[0])
# Creamos un conjunto de datos combinado
datos_entrenamiento_total = torch.cat((
    datos_entrenamiento,
    datos_entrenamiento_transformados_zoom,
    datos_entrenamiento_transformados_translacionX,
    datos_entrenamiento_transformados_translacionY,
    datos_entrenamiento_transformados_flip), dim=0)

# Creamos un conjunto de etiquetas combinado
etiquetas_entrenamiento_total = torch.cat((
    etiquetas_entrenamiento,
    etiquetas_entrenamiento_transformados_zoom,
    etiquetas_entrenamiento_transformados_translacionX,
    etiquetas_entrenamiento_transformados_translacionY,
    etiquetas_entrenamiento_transformados_flip), dim=0)

# Creamos un conjunto de datos combinado
datos_validacion_total = torch.cat((
    datos_validacion,
    datos_validacion_transformados_zoom,
    datos_validacion_transformados_translacionX,
    datos_validacion_transformados_translacionY,
    datos_validacion_transformados_flip), dim=0)

# Creamos un conjunto de etiquetas combinado
etiquetas_validacion_total = torch.cat((
    etiquetas_validacion,
    etiquetas_validacion_transformados_zoom,
    etiquetas_validacion_transformados_translacionX,
    etiquetas_validacion_transformados_translacionY,
    etiquetas_validacion_transformados_flip), dim=0)

# Creamos un conjunto de datos combinado
datos_prueba_total = torch.cat((
    datos_prueba,
    datos_prueba_transformados_zoom,
    datos_prueba_transformados_translacionX,
    datos_prueba_transformados_translacionY,
    datos_prueba_transformados_flip), dim=0)

# Creamos un conjunto de etiquetas combinado
etiquetas_prueba_total = torch.cat((
    etiquetas_prueba,
    etiquetas_prueba_transformados_zoom,
    etiquetas_prueba_transformados_translacionX,
    etiquetas_prueba_transformados_translacionY,
    etiquetas_prueba_transformados_flip), dim=0)

print('ENTRENAMIENTO INICIAL',datos_entrenamiento_total.shape[0])
print('VALIDACION INICIAL',datos_validacion_total.shape[0])
print('PRUEBA INICIAL',datos_prueba_total.shape[0])

ENTRENAMIENTO INICIAL 10770
VALIDACION INICIAL 2308
PRUEBA INICIAL 2310
ENTRENAMIENTO INICIAL 53850
VALIDACION INICIAL 11540
PRUEBA INICIAL 11550


In [ ]:
print(best_params)

In [ ]:
# Concatenar los datos de entrenamiento y validación
datos_entrenamiento_final_total = torch.cat((datos_entrenamiento_total, datos_validacion_total), dim=0)
etiquetas_entrenamiento_final_total = torch.cat((etiquetas_entrenamiento_total, etiquetas_validacion_total), dim=0)

# Preparar los dataloaders con los datos combinados
train_loader, val_loader = preparar_entrenamiento(datos_entrenamiento_final_total, etiquetas_entrenamiento_final_total, datos_prueba_total, etiquetas_prueba_total, 64)

# Entrenamiento del modelo con los mejores parámetros
#best_params = study.best_trial.params
best_params_local = {'learning_rate': 8.492389446863165e-06, 'lstm_units': 1226, 'dense_units': 1094, 'dropout_rate': 0.2860955821910263}
modelo_final = entrenar_modelo(train_loader, val_loader, best_params_local, num_epochs=200)
log_dir = "./logs/fino_transform"
guardar_logs(study, log_dir)


Final Training:  24%|██▍       | 49/200 [35:08<1:46:51, 42.46s/epoch]